A collection of random Pandas problems from stackoverflow, solved using sqltables.

In [2]:
import sqltables

In [3]:
# https://stackoverflow.com/questions/64793902/python-merge-several-columns-of-a-dataframe-without-having-duplicates-of-data
Name = ['Lolo', 'Mike', 'Tobias','Luke','Sam']
Age = [19, 34, 13, 45, 52]
Info_1 = ['Tall', 'Large', 'Small', 'Small','']
Info_2 = ['New York', 'Paris', 'Lisbon', '', 'Berlin']
Info_3 = ['Tall', 'Paris', 'Hi', 'Small', 'Thanks']
Data = [123,268,76,909,87]
Sex = ['F', 'M', 'M','M','M']

columns = {'Name' : Name, 'Age' : Age, 'Info_1' : Info_1, 'Info_2' : Info_2, 'Info_3' : Info_3, 'Data' : Data, 'Sex' : Sex}

In [4]:
db = sqltables.Database()
tab = db.create_table(rows=zip(*columns.values()), column_names=columns.keys())
tab

|Name|Age|Info\_1|Info\_2|Info\_3|Data|Sex|
|-|-|-|-|-|-|-|
|\'Lolo\'|19|\'Tall\'|\'New York\'|\'Tall\'|123|\'F\'|
|\'Mike\'|34|\'Large\'|\'Paris\'|\'Paris\'|268|\'M\'|
|\'Tobias\'|13|\'Small\'|\'Lisbon\'|\'Hi\'|76|\'M\'|
|\'Luke\'|45|\'Small\'|\'\'|\'Small\'|909|\'M\'|
|\'Sam\'|52|\'\'|\'Berlin\'|\'Thanks\'|87|\'M\'|


In [5]:
info_tab = tab \
    .view("select Name, Info_1 as Info from _ union select Name, Info_2 as Info from _ union select Name, Info_3 as Info from _") \
    .view("select distinct * from _") \
    .view("select Name, group_concat(Info, ' ') as Info from _ where Info != '' group by Name")
tab.view("select _.*, coalesce(info.Info, '') as Info from _ left join info on _.Name = info.Name",
        bindings={"info": info_tab})

|Name|Age|Info\_1|Info\_2|Info\_3|Data|Sex|Info|
|-|-|-|-|-|-|-|-|
|\'Lolo\'|19|\'Tall\'|\'New York\'|\'Tall\'|123|\'F\'|\'New York Tall\'|
|\'Mike\'|34|\'Large\'|\'Paris\'|\'Paris\'|268|\'M\'|\'Large Paris\'|
|\'Tobias\'|13|\'Small\'|\'Lisbon\'|\'Hi\'|76|\'M\'|\'Hi Lisbon Small\'|
|\'Luke\'|45|\'Small\'|\'\'|\'Small\'|909|\'M\'|\'Small\'|
|\'Sam\'|52|\'\'|\'Berlin\'|\'Thanks\'|87|\'M\'|\'Berlin Thanks\'|


In [6]:
# https://stackoverflow.com/questions/64793653/how-to-convert-a-dataframe-to-ndarray-of-0s-and-1s
example = """
col_1 col_2
a     4
a     3
b     2
c     2
d     1
b     4
c     1
"""
data = [row.split() for row in example.split("\n") if row != ""][1:]
db = sqltables.Database()
tab = db.create_table(rows=data, column_names=["col_1", "col_2"])
tab

|col\_1|col\_2|
|-|-|
|\'a\'|\'4\'|
|\'a\'|\'3\'|
|\'b\'|\'2\'|
|\'c\'|\'2\'|
|\'d\'|\'1\'|
|\'b\'|\'4\'|
|\'c\'|\'1\'|


In [7]:
crosstab = tab.view("select * from (select distinct col_1 from _), (select distinct col_2 from _)")
counts = tab.view("""
select 
  crosstab.col_1, crosstab.col_2, count(_.col_1) as count 
from crosstab left join _ using (col_1, col_2) group by crosstab.col_1, crosstab.col_2
""", bindings={"crosstab": crosstab})
counts

|col\_1|col\_2|count|
|-|-|-|
|\'a\'|\'1\'|0|
|\'a\'|\'2\'|0|
|\'a\'|\'3\'|1|
|\'a\'|\'4\'|1|
|\'b\'|\'1\'|0|
|\'b\'|\'2\'|1|
|\'b\'|\'3\'|0|
|\'b\'|\'4\'|1|
|\'c\'|\'1\'|1|
|\'c\'|\'2\'|1|
|\'c\'|\'3\'|0|
|\'c\'|\'4\'|0|
|\'d\'|\'1\'|1|
|\'d\'|\'2\'|0|
|\'d\'|\'3\'|0|
|\'d\'|\'4\'|0|


In [8]:
import json

class Json_group_array:
    def __init__(self):
        self.array = []
        
    def step(self, x):
        self.array.append(x)
        
    def finalize(self):
        return json.dumps(self.array)
    
db._conn.create_aggregate("json_group_array", 1, Json_group_array)

In [9]:
[json.loads(x) for [x] in counts.view("select json_group_array(count) from _ group by col_1 order by col_1, col_2")]

[[0, 0, 1, 1], [0, 1, 0, 1], [1, 1, 0, 0], [1, 0, 0, 0]]

In [10]:
# https://stackoverflow.com/questions/64793652/count-top-most-frequent-phrases-in-a-text-column-in-pandas
import re
example = """
Andy | max min | tea | pal
no limit | toy 2011 | hess | mix
Andy | Andy | toy 2011| pal
"""
db = sqltables.Database()
tab = db.create_table(rows=([x] for x in example.split("\n")[1:-1]), column_names=["text"])
tab

|text|
|-|
|\'Andy \| max min \| tea \| pal\'|
|\'no limit \| toy 2011 \| hess \| mix\'|
|\'Andy \| Andy \| toy 2011\| pal\'|


In [11]:
tab2 = db.load_values(([x] for [text] in tab for x in re.split(r"\s*\|\s*", text)), column_names=["text1"])
tab2.view("select text1, count(*) as count from _ group by text1 order by count desc limit 3")

|text1|count|
|-|-|
|\'Andy\'|3|
|\'toy 2011\'|2|
|\'pal\'|2|


In [12]:
# https://stackoverflow.com/questions/64809403/change-the-value-of-a-column-based-on-finding-characters-in-another-column-with
example = """
City - Country
Saddle(Canada) - Other
Dublin - Other
Detroit - USA
Vancouver - Canada
NYC: US - Other
"""
[header, *data] = example.split("\n")[1:-1]
column_names = header.split(" - ")
rows = [x.split(" - ") for x in data]
db = sqltables.Database()
import re, sqlite3
sqlite3.enable_callback_tracebacks(True)
db._conn.create_function("regexp", 2, lambda x, y: bool(re.search(x, y)))
tab = db.create_table(rows=rows, column_names=column_names)
tab

|City|Country|
|-|-|
|\'Saddle\(Canada\)\'|\'Other\'|
|\'Dublin\'|\'Other\'|
|\'Detroit\'|\'USA\'|
|\'Vancouver\'|\'Canada\'|
|\'NYC\: US\'|\'Other\'|


In [13]:
tab.view("select City, regexp('US', City) from _")

|City|regexp\(\'US\'\, City\)|
|-|-|
|\'Saddle\(Canada\)\'|0|
|\'Dublin\'|0|
|\'Detroit\'|0|
|\'Vancouver\'|0|
|\'NYC\: US\'|1|


In [14]:
overrides = db.load_values([
    [0, 'Canada', 'Canada'],
    [1, 'US', 'USA'],
    [2, None, None]
], column_names=["priority", "regex", "Country"])
overrides

|priority|regex|Country|
|-|-|-|
|0|\'Canada\'|\'Canada\'|
|1|\'US\'|\'USA\'|
|2|None|None|


In [15]:
tab.view("select min(priority), City, coalesce(overrides.Country, _.Country) as Country from _, overrides where (overrides.regex is not null and regexp(overrides.regex, _.City)) or overrides.regex is null group by City order by priority", bindings={"overrides": overrides})

|min\(priority\)|City|Country|
|-|-|-|
|0|\'Saddle\(Canada\)\'|\'Canada\'|
|1|\'NYC\: US\'|\'USA\'|
|2|\'Detroit\'|\'USA\'|
|2|\'Dublin\'|\'Other\'|
|2|\'Vancouver\'|\'Canada\'|
